In [97]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
import copy
from functools import partial

In [98]:
# Solves quadratic programming problem of type
# minimize 1/2 * x.T * G * x + a.T * x
# subject to C * x >= b
# Pass G - NxN; a - 1xN; C - MxN; b - 1xM
# return - 1xN
def solve_qp(G, a, C, b):
    x0 = np.random.randn(len(G))

    def loss(x, sign=1.):
        return sign * (0.5 * np.dot(x.T, np.dot(G, x)) + np.dot(a, x))

    def jac(x, sign=1.):
        return sign * (np.dot(x.T, G) + a)

    cons = {
        'type': 'ineq',
        'fun': lambda x: np.dot(C, x) - b,
        'jac': lambda x: C
    }

    opt = {'disp': False}

    return scipy.optimize.minimize(
        loss, x0, jac=jac, constraints=cons, method='SLSQP', options=opt).x

In [99]:
def target_function(x):
    if len(x) > 2:
        x = x[1:]
    return np.sign(x[1] - x[0] + 0.25 * np.sin(np.pi * x[0]))


# returns X Nxd+1 and Y 1xN
def generate_points(target, n_points, r_dimension, bounds, bias=False):
    X = np.random.uniform(bounds[0], bounds[1], (n_points, r_dimension))
    Y = [target(i) for i in X]
    if bias == True:
        X = np.hstack((np.ones((n_points, 1)), X))
    return X, Y


def kernel_rbf(gamma, x1, x2):
    return np.exp(-gamma * np.linalg.norm(np.subtract(x1, x2))**2)

In [135]:
class Hard_svm:
    def __init__(self, X, Y, X_test, Y_test, kernel=None):
        self.X = X
        self.Y = Y
        self.X_test = X_test
        self.Y_test = Y_test
        self.kernel = kernel
    
    def test_e_out(self):
        errors = 0
        for i in range(len(self.X_test)):
            answer = np.sign(self.query(self.X_test[i]))
            if answer != self.Y_test[i]:
                errors+=1
        self.e_out = errors / len(self.X_test)
    
    def test_e_in(self):
        errors = 0
        for i in range(len(self.X)):
            answer = np.sign(self.query(self.X[i]))
            if answer != self.Y[i]:
                errors += 1
        self.e_in = errors / len(self.X)

    def train(self):
        G = self.construct_G()
        a = self.construct_a()
        C = self.construct_C()
        b = self.construct_b()

        self.alphas = np.around(solve_qp(G, np.negative(a), C, b), 9)
        self.alphas = self.make_dense_dict(self.alphas)

        if self.kernel == None:
            self.construct_weights()
        self.construct_bias()

    def query(self, x):
        if self.kernel == None:
            return np.dot(self.w, x) + self.b
        else:
            sum_ = 0.
            for key, value in self.alphas.items():
                sum_ += self.Y[key] * value * self.kernel(self.X[key], x)
            return sum_ + self.b

    def construct_bias(self):
        self.b = 0.
        if len(self.alphas) != 0:
            support_idx = next(iter(self.alphas))
            for key, value in self.alphas.items():
                if self.kernel == None:
                    self.b += self.Y[key] * value * np.dot(
                        self.X[key], self.X[support_idx])
                else:
                    self.b += self.Y[key] * value * self.kernel(
                        self.X[key], self.X[support_idx])
            self.b = self.Y[support_idx] - self.b

    def construct_weights(self):
        self.w = 0.
        for key, value in self.alphas.items():
            self.w += self.Y[key] * value * self.X[key]

    def make_dense_dict(self, array):
        dense_dict = {}
        for i in range(len(array)):
            if array[i] != 0:
                dense_dict[i] = array[i]
        return dense_dict

    def construct_G(self):
        G = []
        for i in range(len(self.Y)):
            g_row = []
            for j in range(len(self.Y)):
                if self.kernel != None:
                    g_row.append(self.Y[i] * self.Y[j] * self.kernel(
                        self.X[i], self.X[j]))
                else:
                    g_row.append(
                        self.Y[i] * self.Y[j] * np.dot(self.X[i], self.X[j]))
            G.append(g_row)
        G = np.reshape(G, (len(self.Y), len(self.Y)))
        return G

    def construct_a(self):
        return np.ones(len(self.Y))

    def construct_C(self):
        C = [self.Y, np.negative(self.Y)]
        C = np.append(C, np.eye(len(self.Y)))
        C = np.reshape(C, (len(self.Y) + 2, -1))
        return C

    def construct_b(self):
        return np.zeros(len(self.Y) + 2)

In [130]:
class Lloyds_algorithm:
    # X - points, K - number of clusters
    def __init__(self, X, K):
        self.X = X
        self.K = K
        self.N = len(self.X)
        
    def lloyds_algorithm(self):
        self.cluster_points = {i: [] for i in range(self.K)}
        self.cluster_centers = np.random.uniform(-1, 1,
                                                 (self.K, len(self.X[0]) - 1))
        self.cluster_centers = np.hstack((np.ones((self.K, 1)),
                                          self.cluster_centers))
        old_norm = float('inf')
        old_centers = np.zeros((self.K, len(self.X[0])))
        new_norm = 0.
        while np.fabs(new_norm - old_norm) >= 1e-9:
            old_norm = new_norm
            old_centers = copy.copy(self.cluster_centers)
            if self.lloyds_algorithm_closest_points() == False:
                return False
            self.lloyds_algorithm_update_centers()
            new_norm = np.linalg.norm(
                np.subtract(self.cluster_centers, old_centers))
        return True

    def lloyds_algorithm_update_centers(self):
        for cluster_idx in range(self.K):
            median = [0] * len(self.X[0])
            for x_idx in self.cluster_points[cluster_idx]:
                median = np.add(self.X[x_idx], median)
            median = np.divide(median, len(self.cluster_points[cluster_idx]))
            self.cluster_centers[cluster_idx] = median

    def lloyds_algorithm_closest_points(self):
        self.clear_cluster_points()

        for x_idx in range(self.N):
            closest_dist = float('inf')
            closest_idx = -1
            for cluster_idx in range(self.K):
                dist = np.linalg.norm(
                    np.subtract(self.X[x_idx],
                                self.cluster_centers[cluster_idx]))
                if dist < closest_dist:
                    closest_dist = dist
                    closest_idx = cluster_idx
            self.cluster_points[closest_idx].append(x_idx)
        if self.count_nonzero_clusters() != self.K:
            return False
        return True

    def clear_cluster_points(self):
        self.cluster_points = {i: [] for i in range(self.K)}

    def count_nonzero_clusters(self):
        non_zero = 0
        for key, value in self.cluster_points.items():
            if len(value) != 0:
                non_zero += 1
        return non_zero

In [176]:
# 13
tests = 300
n_points = 100
gamma = 1.5
kernel = functools.partial(kernel_rbf, gamma)

e_in_zero = 0
for i in range(tests):
    X, Y = generate_points(target_function, n_points, 2, [-1, 1], True)
    svm_rbf = Hard_svm(X, Y,[],[], kernel)
    svm_rbf.train()
    svm_rbf.test_e_in()
    if svm_rbf.e_in == 0:
        e_in_zero+=1
print('E_in = 0: ',e_in_zero / tests * 100,'% of times')

E_in = 0:  99.33333333333333 % of times


In [133]:
class Regular_form_rbf:
    def __init__(self, X, Y, X_test, Y_test, gamma, K):
        self.X = X
        self.Y = Y
        self.X_test =X_test
        self.Y_test = Y_test
        self.gamma = gamma
        self.K = K
        self.N = len(self.X)
        self.lloyd = Lloyds_algorithm(self.X, self.K)

    def feature_transform_point(self, x):
        z_row = [1]
        for cluster_idx in range(self.K):
            z_row.append(
                self.feature_transform(x, self.cluster_centers[cluster_idx]))
        return z_row

    def feature_transform(self, x, mu):
        return np.exp(-0.5 * np.power(
            self.gamma * np.linalg.norm(np.subtract(x, mu)), 2))

    def construct_feature_matrix(self, X):
        Z = []
        for x_idx in range(len(X)):
            Z.append(self.feature_transform_point(X[x_idx]))
        return np.reshape(Z, (len(X), -1))

    def train(self):
        if self.lloyd.lloyds_algorithm():
            # Run Lloyd's algorithm to find clusters
            self.cluster_centers = self.lloyd.cluster_centers
            self.cluster_points = self.lloyd.cluster_points

            # Run hard SVM to find weights and final hypothesis
            self.Z = self.construct_feature_matrix(self.X)
            self.Z_test = self.construct_feature_matrix(self.X_test)
            self.hard_svm = Hard_svm(self.Z, self.Y, self.Z_test, self.Y_test)
            self.hard_svm.train()
            return True
        return False

    def query(self, x):
        z = self.feature_transform_point(x)
        return np.sign(self.hard_svm.query(z))

    def test_e_in(self):
        self.hard_svm.test_e_in()
        self.e_in = self.hard_svm.e_in
        
    def test_e_out(self):
        self.hard_svm.test_e_out()
        self.e_out = self.hard_svm.e_out

In [173]:
def plot_clusters(regular_rbf):
    for idx in range(regular_rbf.K):
        rgb = np.random.randint(0,256,3)/255
        for x_idx in regular_rbf.cluster_points[idx]:
            plt.plot(
                regular_rbf.X[x_idx][1],
                regular_rbf.X[x_idx][2],
                color=tuple(rgb),
                marker='.')
        plt.plot(
            regular_rbf.cluster_centers[idx][1],
            regular_rbf.cluster_centers[idx][2],
            color=tuple(rgb),
            marker='*',
            markersize=10,
        )
    plt.show()

In [158]:
# 14, 15, 16
tests = 200
n_points = 100
n_test_points = 1000
gamma = 1.5
K = [9,12]
kernel = functools.partial(kernel_rbf, gamma)

for k in K:
    regular_rbf_total_e_in = 0.
    regular_rbf_total_e_out = 0.
    svm_rbf_total_e_in = 0.
    svm_rbf_total_e_out = 0.
    n_svm_rbf_e_out_better = 0
    for i in range(tests):
        res = False
        X_test, Y_test = generate_points(target_function, n_test_points, 2,
                                         [-1, 1], True)
        while res == False:
            X, Y = generate_points(target_function, n_points, 2, [-1, 1], True)
            regular_rbf = Regular_form_rbf(X, Y, X_test, Y_test, gamma, k)
            res = regular_rbf.train()
        regular_rbf.test_e_in()
        regular_rbf.test_e_out()

        svm_rbf_curr_e_in = 100
        while(svm_rbf_curr_e_in!=0):
            svm_rbf = Hard_svm(X, Y, X_test, Y_test, kernel)
            svm_rbf.train()
            svm_rbf.test_e_in()
            svm_rbf_curr_e_in = svm_rbf.e_in
        svm_rbf.test_e_out()

        if svm_rbf.e_out < regular_rbf.e_out:
            n_svm_rbf_e_out_better += 1

        regular_rbf_total_e_in += regular_rbf.e_in
        regular_rbf_total_e_out += regular_rbf.e_out
        svm_rbf_total_e_in += svm_rbf.e_in
        svm_rbf_total_e_out += svm_rbf.e_out

    print('Results for K: ', k, ' and Gamma: ', gamma, ' over ', tests, ' runs')
    print('Regular RBF average E_in: ', regular_rbf_total_e_in / tests)
    print('Regular RBF average E_out: ', regular_rbf_total_e_out / tests)

    print('Kernel hard SVM average E_in: ', svm_rbf_total_e_in / tests)
    print('Kernel hard SVM average E_out: ', svm_rbf_total_e_out / tests)

    print('Kernel hard SVM had better E_out that resular RBF: ',
          n_svm_rbf_e_out_better / tests * 100, '% of times')
    print('\n')
    
# plot_clusters(regular_rbf)

Results for K:  9  and Gamma:  1.5  over  200  runs
Regular RBF average E_in:  0.05499999999999997
Regular RBF average E_out:  0.07907500000000002
Kernel hard SVM average E_in:  0.0
Kernel hard SVM average E_out:  0.03122499999999999
Kernel hard SVM had better E_out that resular RBF:  80.5 % of times


Results for K:  12  and Gamma:  1.5  over  200  runs
Regular RBF average E_in:  0.06289999999999994
Regular RBF average E_out:  0.08598499999999999
Kernel hard SVM average E_in:  0.0
Kernel hard SVM average E_out:  0.03214999999999999
Kernel hard SVM had better E_out that resular RBF:  75.5 % of times




In [169]:
# 17
tests = 200
n_points = 100
n_test_points = 1000
gamma = [1.5, 2]
K = 9

for g in gamma:
    regular_rbf_total_e_in = 0.
    regular_rbf_total_e_out = 0.
    for i in range(tests):
        res = False
        X_test, Y_test = generate_points(target_function, n_test_points, 2,
                                         [-1, 1], True)
        while res == False:
            X, Y = generate_points(target_function, n_points, 2, [-1, 1], True)
            regular_rbf = Regular_form_rbf(X, Y, X_test, Y_test, g, K)
            res = regular_rbf.train()
        regular_rbf.test_e_in()
        regular_rbf.test_e_out()
        regular_rbf_total_e_in += regular_rbf.e_in
        regular_rbf_total_e_out += regular_rbf.e_out
    print('Results for K: ', K, ' and Gamma: ', g, ' over ', tests, ' runs')
    print('Regular RBF average E_in: ', regular_rbf_total_e_in / tests)
    print('Regular RBF average E_out: ', regular_rbf_total_e_out / tests)
    print('\n')

Results for K:  9  and Gamma:  1.5  over  200  runs
Regular RBF average E_in:  0.0846
Regular RBF average E_out:  0.10505000000000005


Results for K:  9  and Gamma:  2  over  200  runs
Regular RBF average E_in:  0.0554
Regular RBF average E_out:  0.08231999999999993




In [172]:
# 18
tests = 200
n_points = 100
gamma = 1.5
K = 9

e_in_zero = 0
regular_rbf = 0.
for i in range(tests):
    res = False
    while res == False:
        X, Y = generate_points(target_function, n_points, 2, [-1, 1], True)
        regular_rbf = Regular_form_rbf(X, Y, X_test, Y_test, g, K)
        res = regular_rbf.train()
    regular_rbf.test_e_in()
    if regular_rbf.e_in == 0.:
        e_in_zero += 1
print('Results for K: ', K, ' and Gamma: ', gamma, ' over ', tests, ' runs')
print('Regular RBF acieved E_in=0.0 in ', e_in_zero / tests * 100,
      '% of the times')

Results for K:  9  and Gamma:  1.5  over  200  runs
Regular RBF acieved E_in=0.0 in  70.0 % of the times
